In [ ]:
!pip install wtfml==0.0.2
!pip install pretrainedmodels

     |████████████████████████████████| 61kB 6.1MB/s 
  Created wheel for pretrainedmodels: filename=pretrainedmodels-0.7.4-cp36-none-any.whl size=60963 sha256=ad7a768946ac5b53c7e89f2c9f07d426fb26fbae2e1f9fd968ed177025bd065b
  Stored in directory: /root/.cache/pip/wheels/69/df/63/62583c096289713f22db605aa2334de5b591d59861a02c2ecd
Successfully built pretrainedmodels


In [ ]:
!unzip "/content/drive/MyDrive/Para compartir/DeepCancerMelanoma/archive.zip"

Se truncaron las últimas líneas 5000 del resultado de transmisión.
  inflating: train/ISIC_8512716.png  
  inflating: train/ISIC_8513112.png  
  inflating: train/ISIC_8513649.png  
  inflating: train/ISIC_8514170.png  
  inflating: train/ISIC_8514989.png  
  inflating: train/ISIC_8515165.png  
  inflating: train/ISIC_8515281.png  
  inflating: train/ISIC_8515582.png  
  inflating: train/ISIC_8515674.png  
  inflating: train/ISIC_8516181.png  
  inflating: train/ISIC_8516223.png  
  inflating: train/ISIC_8516516.png  
  inflating: train/ISIC_8516631.png  
  inflating: train/ISIC_8516659.png  
  inflating: train/ISIC_8516922.png  
  inflating: train/ISIC_8517305.png  
  inflating: train/ISIC_8517492.png  
  inflating: train/ISIC_8518406.png  
  inflating: train/ISIC_8519222.png  
  inflating: train/ISIC_8519402.png  
  inflating: train/ISIC_8519725.png  
  inflating: train/ISIC_8519890.png  
  inflating: train/ISIC_8519931.png  
  inflating: train/ISIC_8520545.png  
  inflating: train/IS

In [ ]:
import os
import torch
import albumentations

import numpy as np
import pandas as pd

import torch.nn as nn
from sklearn import metrics
from sklearn import model_selection
from torch.nn import functional as F

from wtfml.utils import EarlyStopping
from wtfml.engine import Engine
from wtfml.data_loaders.image import ClassificationLoader

import pretrainedmodels

In [ ]:
class SEResnext50_32x4d(nn.Module):
    def __init__(self, pretrained='imagenet'):
        super(SEResnext50_32x4d, self).__init__()
        
        self.base_model = pretrainedmodels.__dict__[
            "se_resnext50_32x4d"
        ](pretrained=None)
        if pretrained is not None:
            self.base_model.load_state_dict(
                torch.load(
                    "/content/drive/MyDrive/Para compartir/DeepCancerMelanoma/se_resnext50_32x4d-a260b3a4.pth"
                )
            )

        self.l0 = nn.Linear(2048, 1)
    
    def forward(self, image, targets):
        batch_size, _, _, _ = image.shape
        
        x = self.base_model.features(image)
        x = F.adaptive_avg_pool2d(x, 1).reshape(batch_size, -1)
        
        out = self.l0(x)
        loss = nn.BCEWithLogitsLoss()(out, targets.view(-1, 1).type_as(x))

        return out, loss

In [ ]:
# create folds
df = pd.read_csv("/content/drive/MyDrive/Para compartir/DeepCancerMelanoma/train.csv")
df["kfold"] = -1    
df = df.sample(frac=1).reset_index(drop=True)
y = df.target.values
kf = model_selection.StratifiedKFold(n_splits=5)

for f, (t_, v_) in enumerate(kf.split(X=df, y=y)):
    df.loc[v_, 'kfold'] = f

df.to_csv("train_folds.csv", index=False)

In [ ]:
def train(fold):
    training_data_path = "/content/train"
    df = pd.read_csv("/content/train_folds.csv")
    device = "cuda"
    epochs = 3
    train_bs = 32
    valid_bs = 16

    df_train = df[df.kfold != fold].reset_index(drop=True)
    df_valid = df[df.kfold == fold].reset_index(drop=True)

    model = SEResnext50_32x4d(pretrained="imagenet")
    model.to(device)

    mean = (0.485, 0.456, 0.406)
    std = (0.229, 0.224, 0.225)
    train_aug = albumentations.Compose(
        [
            albumentations.Normalize(mean, std, max_pixel_value=255.0, always_apply=True),
            albumentations.ShiftScaleRotate(shift_limit=0.0625, scale_limit=0.1, rotate_limit=15),
            albumentations.Flip(p=0.5)
        ]
    )

    valid_aug = albumentations.Compose(
        [
            albumentations.Normalize(mean, std, max_pixel_value=255.0, always_apply=True)
        ]
    )

    train_images = df_train.image_name.values.tolist()
    train_images = [os.path.join(training_data_path, i + ".png") for i in train_images]
    train_targets = df_train.target.values

    valid_images = df_valid.image_name.values.tolist()
    valid_images = [os.path.join(training_data_path, i + ".png") for i in valid_images]
    valid_targets = df_valid.target.values

    train_dataset = ClassificationLoader(
        image_paths=train_images,
        targets=train_targets,
        resize=None,
        augmentations=train_aug,
    )

    train_loader = torch.utils.data.DataLoader(
        train_dataset, batch_size=train_bs, shuffle=True, num_workers=4
    )

    valid_dataset = ClassificationLoader(
        image_paths=valid_images,
        targets=valid_targets,
        resize=None,
        augmentations=valid_aug,
    )

    valid_loader = torch.utils.data.DataLoader(
        valid_dataset, batch_size=valid_bs, shuffle=False, num_workers=4
    )

    optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)
    scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(
        optimizer,
        patience=3,
        threshold=0.001,
        mode="max"
    )

    es = EarlyStopping(patience=5, mode="max")

    for epoch in range(epochs):
        train_loss = Engine.train(train_loader, model, optimizer, device=device)
        predictions, valid_loss = Engine.evaluate(
            valid_loader, model, device=device
        )
        predictions = np.vstack((predictions)).ravel()
        auc = metrics.roc_auc_score(valid_targets, predictions)
        print(f"Epoch = {epoch}, AUC = {auc}")
        scheduler.step(auc)

        es(auc, model, model_path=f"model_fold_{fold}.bin")
        if es.early_stop:
            print("Early stopping")
            break

In [ ]:
def predict(fold):
    test_data_path = "/content/test"
    df = pd.read_csv("/content/drive/MyDrive/Para compartir/DeepCancerMelanoma/test.csv")
    device = "cuda"
    model_path=f"model_fold_{fold}.bin"

    mean = (0.485, 0.456, 0.406)
    std = (0.229, 0.224, 0.225)
    aug = albumentations.Compose(
        [
            albumentations.Normalize(mean, std, max_pixel_value=255.0, always_apply=True)
        ]
    )

    images = df.image_name.values.tolist()
    images = [os.path.join(test_data_path, i + ".png") for i in images]
    targets = np.zeros(len(images))

    test_dataset = ClassificationLoader(
        image_paths=images,
        targets=targets,
        resize=None,
        augmentations=aug,
    )

    test_loader = torch.utils.data.DataLoader(
        test_dataset, batch_size=16, shuffle=False, num_workers=4
    )

    model = SEResnext50_32x4d(pretrained=None)
    model.load_state_dict(torch.load(model_path))
    model.to(device)

    predictions = Engine.predict(test_loader, model, device=device)
    predictions = np.vstack((predictions)).ravel()

    return predictions

In [ ]:
train(0)
train(1)
train(2)
train(3)
train(4)


100%|██████████| 829/829 [02:42<00:00,  5.10it/s, loss=0.0811]

100%|██████████| 415/415 [00:19<00:00, 21.00it/s, loss=0.072]


Epoch = 0, AUC = 0.8738170554117705
Validation score improved (-inf --> 0.8738170554117705). Saving model!



100%|██████████| 829/829 [02:42<00:00,  5.10it/s, loss=0.0667]

100%|██████████| 415/415 [00:19<00:00, 21.01it/s, loss=0.0663]


Epoch = 1, AUC = 0.8940940420430357
Validation score improved (0.8738170554117705 --> 0.8940940420430357). Saving model!



100%|██████████| 829/829 [02:42<00:00,  5.10it/s, loss=0.0617]

100%|██████████| 415/415 [00:19<00:00, 21.05it/s, loss=0.0639]


...

Epoch = 0, AUC = 0.8363578811301063
Validation score improved (-inf --> 0.8363578811301063). Saving model!



100%|██████████| 829/829 [02:41<00:00,  5.12it/s, loss=0.0671]

100%|██████████| 415/415 [00:18<00:00, 22.38it/s, loss=0.0718]


Epoch = 1, AUC = 0.8664938732047405
Validation score improved (0.8363578811301063 --> 0.8664938732047405). Saving model!



100%|██████████| 829/829 [02:42<00:00,  5.11it/s, loss=0.0612]

100%|██████████| 415/415 [00:18<00:00, 22.60it/s, loss=0.0684]


Epoch = 2, AUC = 0.8736179613373525
Validation score improved (0.8664938732047405 --> 0.8736179613373525). Saving model!


In [ ]:
p1 = predict(0)
p2 = predict(1)
p3 = predict(2)
p4 = predict(3)
p5 = predict(4)


100%|██████████| 687/687 [00:27<00:00, 25.44it/s]

100%|██████████| 687/687 [00:27<00:00, 25.13it/s]

100%|██████████| 687/687 [00:27<00:00, 25.27it/s]

100%|██████████| 687/687 [00:26<00:00, 25.46it/s]

100%|██████████| 687/687 [00:27<00:00, 25.11it/s]


In [ ]:
predictions = (p1 + p2 + p3 + p4 + p5) / 5
sample = pd.read_csv("/content/drive/MyDrive/Para compartir/DeepCancerMelanoma/sample_submission.csv")
sample.loc[:, "target"] = predictions
sample.to_csv("submission.csv", index=False)